---
---
# **ALGORITMA KONSISTENSI SSKI**
---
---

---
### **IMPORT LIBRARIES AND OTHER DEPENDANCIES**
---

In [30]:
import os
import io
import glob
import time
import datetime
import numpy as np
import pandas as pd
import openpyxl # type: ignore
from openpyxl import load_workbook, Workbook # type: ignore
from openpyxl.styles import (Border, Side, Alignment, Font, PatternFill)
from openpyxl.utils import get_column_letter # type: ignore
from openpyxl.worksheet.table import Table, TableStyleInfo # type: ignore
from collections import Counter
import re
import string
from copy import copy
from IPython.display import display, HTML
import streamlit as st # type: ignore
import json
import datetime

In [31]:
def mapping_dictionary(wb_path):
    wb = openpyxl.load_workbook(wb_path, data_only=True)
    sheet = wb['Dictionary Intra Antar']
    mappings = {}
    for row in sheet.iter_rows(min_row=2, max_col=3, values_only=True):
        key = str(row[0])  # Kolom A berisi key di file dictionary (fixed)
        description = str(row[1])  # Kolom B berisi deskripsi di file dictionary (fixed)
        description_list = [desc.strip() for desc in
                            description.split(
                                ';')]  # Ini untuk membaca deksripsinya menjadi sebuah list jika ada ','
        mappings[key] = description_list
    return mappings

In [32]:
def find_key_by_value(mappings, target_value):
    for key, value in mappings.items():
        if value[0] == target_value:
            return key
    return None

In [33]:
file_path = "/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/Sumber_Data_Lama/SSKI/SSKI EKSTERNAL_25 Okt 2024.xlsx"
super_sheet_old = pd.ExcelFile("/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/SSKI_OKTOBER_2024.xlsx")
super_sheet_new = pd.ExcelFile("/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/SSKI EKSTERNAL_November 2024 (1).xlsx")
sski_folder = r'/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/Sumber_Data_Lama/SSKI'
wb_path = "/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/Kodifikasi SSKI_v1.xlsx"

In [34]:
# Extract the filename from the path
file_name = file_path.split("/")[-1]

# Use regular expressions to extract the month and year
match = re.search(r"([A-Za-z]+) (\d{4})", file_name)
if match:
    month = match.group(1)  # Extract the month
    year = match.group(2)    # Extract the year


mappings = mapping_dictionary(wb_path)

if os.sep == "/":
    nama_sski_folder = sski_folder.split('/')[-1]
    filenames_sski = glob.glob(sski_folder + r"/*.xlsx")
elif os.sep == "\\":
    nama_sski_folder = sski_folder.split('\\')[-1]
    filenames_sski = glob.glob(sski_folder + r"\*.xlsx")
workbooks_sski = {}

In [35]:
print(filenames_sski)

['/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/Sumber_Data_Lama/SSKI/SSKI EKSTERNAL_25 Okt 2024.xlsx', '/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/Sumber_Data_Lama/SSKI/SSKI_SEPTEMBER_2024.xlsx', '/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/Sumber_Data_Lama/SSKI/~$SSKI EKSTERNAL_25 Okt 2024 new.xlsx']


In [36]:
for filename in filenames_sski:
    # Skip file sementara Excel yang dimulai dengan ~$ 
    if '~$' in filename:
        continue
    
    wb_sski = openpyxl.load_workbook(filename)
    sheet_names_sski = wb_sski.sheetnames
    # print(sheet_names_sski)
    for sheet_name in sheet_names_sski:
        sheet_sski = wb_sski[sheet_name]
        
        # Hilangkan semua tanda titik dari nama sheet
        sheet_name_clean = sheet_name.replace('.', '')
        
        # Dapatkan indeks sheet menggunakan find_key_by_value
        sheet_index_sski = find_key_by_value(mappings, sheet_name_clean)
        
        # Pastikan indeks sheet adalah string
        if sheet_index_sski is None:
            sheet_index_sski = sheet_name_clean  # Atau tangani sesuai kebutuhan
        else:
            sheet_index_sski = str(sheet_index_sski)
        
        # Buat nama sheet yang sudah dimodifikasi
        modified_sheet_name_sski = f'{nama_sski_folder}{sheet_index_sski}'
        
        # Tambahkan sheet ke dictionary globals
        globals()[modified_sheet_name_sski] = sheet_sski

In [37]:
def indikator(wb_path, mappings):
    wb = openpyxl.load_workbook(wb_path, data_only=True)
    sheet = wb['Kodifikasi Vertical Check']

    kelompok = []
    indikator_asli = []
    indikator_calculated = []

    last_row = sheet.max_row
    four_digit_keys = [key for key in mappings.keys() if (len(key) == 4 or len(key) == 5)]

    for row in sheet.iter_rows(min_row=2):
        # Check if column 2 contains 'SSKI'
        if 'SSKI' not in str(row[1].value):
            continue  # Skip rows where column 2 does not contain 'SSKI'
        else:
            new_indikator_asli = str(row[1].value)
            new_indikator_calculated = str(row[2].value)
            kelompok.append(row[0].value)
            indikator_asli.append(new_indikator_asli)
            indikator_calculated.append(new_indikator_calculated)

    # Return the results
    return kelompok, indikator_asli, indikator_calculated

In [38]:
mappings = mapping_dictionary(wb_path)
kelompok, indikator_asli, indikator_calculated = indikator(wb_path, mappings)

In [39]:
def dataframe_bag(indikator_bag, mappings):
    queries = re.split(r'[+\-;/]', indikator_bag)
    grup_data = []
    list_data = []
    paths = []
    indikator_name=[]
    
    for query in queries:
        token = query.split('.')
        paths.append(query)
        data = token[0] + token[1]
        grup_data.append(data)

    if grup_data:
        list_data = [globals().get(name) for name in grup_data if name in globals()]
    
    token = queries[0].split('.')
    
    results = []

    for ws, query in zip(list_data, queries):
        data_frames = []
        token = query.split('.')
        baris_pembukaan = 1
        nama_indikator = []
        last_found_column_idx = 0

        for i, t in enumerate(token[2:]):
            if t in mappings:
                deskripsi_list = mappings[t]
                ditemukan = False

                for deskripsi in deskripsi_list:
                    desk=deskripsi
                    deskripsi = deskripsi.replace('\u2011', '-').replace('\u00A0', ' ').strip().replace(' ', '').lower()
                    
                    for baris in range(baris_pembukaan, ws.max_row + 1):
                        for kolom_idx in range(last_found_column_idx, len(string.ascii_uppercase)):
                            kolom = string.ascii_uppercase[kolom_idx]
                            nilai_sel = ws[f'{kolom}{baris}'].value
                            if nilai_sel is not None:
                                nilai_sel = str(nilai_sel).replace('\u2011', '-').replace('\u00A0', ' ').strip().replace(' ', '').lower()
                                if nilai_sel == '-':
                                    nilai_sel = 0

                            if nilai_sel == deskripsi:
                                baris_pembukaan = baris
                                nama_indikator.append((t, nilai_sel, baris))
                                ditemukan = True
                                last_found_column_idx = kolom_idx
                                break
                        if ditemukan:
                            break
                    if ditemukan:
                        break

        indikator_name.append(desk)
        kolom_kodifikasi = kolom
        kolom_nilai = ord(kolom_kodifikasi.upper()) - ord('A') + 2
        results.append((nama_indikator, baris_pembukaan))
    
    baris_pembukaan_list = [hasil[1] for hasil in results]

    def create_data_frames(ws, baris_data, baris_kolom):
        # Set header rows based on the sheet name
        if ws.title in ['11', '16']:
            header_rows = [3, 4, 5, 6]
        elif ws.title == '5a':
            header_rows = [4, 5, 6]
            header_data = []
            for row in header_rows:
                header_data.append([ws.cell(row=row, column=col).value for col in range(2, ws.max_column + 1)])
            for row in [0, 1]:  # Loop through both header_data[1] and header_data[2]
                for i in range(1, len(header_data[row])):
                    if header_data[row][i] is None:
                        header_data[row][i] = header_data[row][i - 1]
        elif ws.title in ['5b', '5c', '5d', '5d.1', '5.d.2']:
            header_rows = [3, 4, 5, 6]
            header_data = []
            for row in header_rows:
                header_data.append([ws.cell(row=row, column=col).value for col in range(2, ws.max_column + 1)])
            for row in [0, 1, 2]:  # Loop through both header_data[1] and header_data[2]
                for i in range(1, len(header_data[row])):
                    if header_data[row][i] is None:
                        header_data[row][i] = header_data[row][i - 1]
        else:
            header_rows = [4, 5, 6]
            header_data = []
            for row in header_rows:
                header_data.append([ws.cell(row=row, column=col).value for col in range(2, ws.max_column + 1)])
            for row in [1]:  # Loop through both header_data[1] and header_data[2]
                for i in range(1, len(header_data[row])):
                    if header_data[row][i] is None:
                        header_data[row][i] = header_data[row][i - 1]
    
        column_names = [' '.join(str(cell) for cell in col if cell is not None) for col in zip(*header_data)]
        column_names = [str(name).upper() if 'Unnamed' in str(name) else name for name in column_names]
        column_names = [name.strip() if isinstance(name, str) else name for name in column_names]
        column_names = [f'{name[0]}-{name[1].capitalize()}' if isinstance(name, tuple) else name for name in column_names]
    
        data = []
        for col in range(2, ws.max_column + 1):
            cell_value = ws.cell(row=baris_data, column=col).value
            if cell_value is None or cell_value == '-' or str(cell_value).strip() == '':
                cell_value = 0
            data.append(cell_value)
    
        df = pd.DataFrame([data], columns=column_names)
        df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
        return df
    
    data_frames_dict = {}
    
    for idx, (ws, baris_data) in enumerate(zip(list_data, baris_pembukaan_list)):
        baris_kolom = None
        for i in range(3, 8):
            df_1 = create_data_frames(ws, baris_data, i)
            df_1 = df_1.drop(df_1.columns[[0]], axis=1)
            if not df_1.empty:
                first_col = df_1.columns[0]
                if any(isinstance(first_col, int) and 1900 <= first_col <= 2100 for first_col in df_1.columns):
                    baris_kolom = i
                    break
        
        df = create_data_frames(ws, baris_data, baris_kolom)
        df = df.iloc[:, 1:-2]
        data_frames_dict[(ws, idx)] = (baris_kolom, df)
        kolom = df.columns.to_list()

    data_frames = []
    
    for (ws, idx), (baris_kolom, df) in data_frames_dict.items():
        df = df.drop(df.columns[[0]], axis=1)
        data_frames.append(df)
    
    combined_df = pd.concat(data_frames, ignore_index=True)
    combined_df['Indikator'] = indikator_name  # Insert 'Indikator' at index 0
    combined_df['Path']= paths
    cols = combined_df.columns.tolist()
    new_order = cols[-2:] + cols[:-2]
    combined_df = combined_df[new_order]

    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    values_by_column = [combined_df.iloc[:, idx].tolist() for idx in range(1, combined_df.shape[1])]
    
    return values_by_column, combined_df

In [40]:
def gabungkan_dataframe_2bag(list_tahun, indikator_asli, indikator_calculated, mappings):
    # Mengolah data dengan fungsi dataframe_bag1
    values_bag1, df1 = dataframe_bag(indikator_asli, mappings)

    # Mengolah data dengan fungsi dataframe_bag2
    values_bag2, df2 = dataframe_bag(indikator_calculated, mappings)
    # display(df1)
    # display(df2)

    # Filter df1 based on list_tahun
    if list_tahun:
        columns_to_keep_bag1 = [df1.columns[0]] + [df1.columns[1]] + [col for col in df1.columns if any(tahun in col for tahun in list_tahun)]
    else:
        columns_to_keep_bag1 = df1.columns.tolist()

    columns_to_keep_bag1 = pd.Index(columns_to_keep_bag1).drop_duplicates().tolist()
    df1_filtered = df1[columns_to_keep_bag1]
    df1_filtered = df1_filtered.loc[:, ~df1_filtered.columns.duplicated()]


    # Filter df2 based on list_tahun
    if list_tahun:
        columns_to_keep_bag2 = [df2.columns[0]] + [df2.columns[1]] + [col for col in df2.columns if any(f'{tahun}' in col for tahun in list_tahun)]
    else:
        columns_to_keep_bag2 = df2.columns.tolist()
    columns_to_keep_bag2 = pd.Index(columns_to_keep_bag2).drop_duplicates().tolist()
    df2_filtered = df2[columns_to_keep_bag2]
    df2_filtered = df2_filtered.loc[:, ~df2_filtered.columns.duplicated()]

    common_columns = df1_filtered.columns.intersection(df2_filtered.columns).tolist()

    df1_filtered_common = df1_filtered[common_columns]
    df2_filtered_common = df2_filtered[common_columns]

    df_gabungan = pd.concat([df1_filtered_common, df2_filtered_common], ignore_index=True)
    df_gabungan = df_gabungan.round(2)

    periode = df_gabungan.columns.tolist()
    date_columns = periode[2:]

    if len(date_columns) == 1:
        periode_range = f"{date_columns[0]}"
    else:
        periode_range = f"{date_columns[0]} - {date_columns[-1]}"

    # Retrieve cleaned values for each source
    values_bag1_clean = [df1_filtered_common[col].tolist() for col in df1_filtered_common.columns]
    values_bag2_clean = [df2_filtered_common[col].tolist() for col in df2_filtered_common.columns]

    # Calculate jumlah_bag1
    jumlah_bag1 = []
    for sublist in values_bag1_clean[2:]:
        if "SSKI.6.RK.AJK.KPI" in indikator_asli or "SSKI.6.SOS.KPIAS" in indikator_asli or "SSKI.3.KPD" in indikator_asli or "SSKI.20.BU20" in indikator_asli or "SSKI.20.BDKU20.BKU20" in indikator_asli or "SSKI.20.BDKU20.BJP20" in indikator_asli or "SSKI.20.RRK" in indikator_asli or "SSKI.20.KUTKU" in indikator_asli or "SSKI.18.IHS18.SPK18" in indikator_asli or indikator_asli=="SSKI.17.KKRTP17" or indikator_asli=="SSKI.17.KPRTL17" or indikator_asli=="SSKI.15.KUP15" or indikator_asli=="SSKI.11a.KA11a" or indikator_asli =="SSKI.9.KPMV9" or indikator_asli=="SSKI.7.KPTA7" or indikator_asli == "SSKI.8.ADP8.KIDP8":
            jumlah_bag1_value = 100
        else:
            jumlah_bag1_value = sum(sublist)
        jumlah_bag1.append(jumlah_bag1_value)

    # Calculate jumlah_bag2
    jumlah_bag2 = []
    for sublist in values_bag2_clean[2:]:
        # Remove both None and NaN values from sublist
        cleaned_sublist = [x for x in sublist if x is not None and pd.notna(x)]

        # Perform calculations based on `indikator_calculated`
        if "-" in indikator_calculated:
            if cleaned_sublist:  # Check if the cleaned sublist is not empty
                jumlah_bag2_value = cleaned_sublist[0] - sum(cleaned_sublist[1:]) if len(cleaned_sublist) > 1 else cleaned_sublist[0]
            else:
                jumlah_bag2_value = 0
        elif ";" in indikator_calculated:
            numeric_elements = [x for x in cleaned_sublist if isinstance(x, (int, float))]
            jumlah_bag2_value = sum(numeric_elements) / len(numeric_elements) if numeric_elements else 0
        elif "/" in indikator_calculated:
            if len(cleaned_sublist) > 1:
                jumlah_bag2_value = (cleaned_sublist[0] / cleaned_sublist[1]) * 100 if cleaned_sublist[1] != 0 else 0
            else:
                jumlah_bag2_value = 0
        else:
            jumlah_bag2_value = sum(cleaned_sublist) if cleaned_sublist else 0
        
        jumlah_bag2.append(jumlah_bag2_value)

    # Calculate `selisih` as the absolute difference between `jumlah_bag1` and `jumlah_bag2`
    selisih = [round(abs(jumlah_bag1[i] - jumlah_bag2[i]), 2) for i in range(len(jumlah_bag1))]

    # Prepare 'selisih' row with 'Selisih' as the first entry
    selisih_row = [None] * len(df_gabungan.columns)
    selisih_row[0] = 'Selisih'

    # Populate `selisih` values starting from the third column
    for idx, value in enumerate(selisih, start=2):
        selisih_row[idx] = value

    # Append the 'Selisih' row to the DataFrame
    df_gabungan.loc[df_gabungan.shape[0]] = selisih_row


    value = df_gabungan.iloc[0, 0]

    # Create a DataFrame containing jumlah_bag1, jumlah_bag2, and selisih
    df_summary = pd.DataFrame({
        df_gabungan.columns[0]: [value, value + ' (Calculated)', 'Selisih'],
    })
    
    for i, col in enumerate(df_gabungan.columns[2:], start=2):
        df_summary[col] = [jumlah_bag1[i-2], jumlah_bag2[i-2], selisih[i-2]]

    df_raw = df_gabungan.copy()
    # Drop columns where the "Selisih" row has a value of 0
    columns_to_drop = df_gabungan.columns[df_gabungan.iloc[-1].isin([0, 100, np.nan])]
    df_gabungan.drop(columns=columns_to_drop, inplace=True)
    df_summary.drop(columns=columns_to_drop, inplace=True)

    return df_gabungan, df_raw, df_summary

In [41]:
def is_numeric(x):
    # Convert to string and strip whitespace
    x = str(x).strip()  
    
    # Remove unwanted characters (like asterisks and tabs)
    x = x.replace('*', '').replace('\t', '').replace(',', '').strip()  # Remove asterisks and tabs, and replace commas
    
    if x == '-':  # Ignore the "-" string
        return True
    if x in ['0', '.']:  # Include valid values
        return True
    try:
        float(x)  # Try to convert to a float
        return True
    except (ValueError, TypeError):  # If conversion fails, it's not a valid number
        return False


In [42]:
def prepare_dataframe(super_sheet ,sheet_name,year_to_check):
    # Load the sheet with dynamic header handling
    input_df = super_sheet.parse(sheet_name, header=[4, 5])

    # Adjust the column names
    input_df.columns = input_df.columns.map(lambda x: str(x[0]).upper() if 'Unnamed' in str(x) else x)
    input_df.columns = input_df.columns.map(lambda x: (x[0], x[1].strip()) if isinstance(x, tuple) and isinstance(x[1], str) else x)

    # Extract the first two columns
    first_column = input_df.iloc[:, 0]
    second_column = input_df.iloc[:, 1]

    # Step 2: Filter columns that have the year in the name (using regex)
    output_df = input_df.filter(regex=str(year_to_check))

    # Reinsert 'NO' and 'Komponen' columns at the beginning of the filtered DataFrame
    output_df.insert(0, 'NO', first_column)
    output_df.insert(1, 'Komponen', second_column)

    # Step 3: Remove rows after 'keterangan' if found
    index_keterangan = output_df[output_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index
    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        output_df = output_df.iloc[:first_keterangan_index]

    # Step 4: Apply numeric validation only to numeric columns
    numeric_columns = output_df.columns[2:]  # Only consider columns after 'NO' and 'Komponen'
    mask_non_numeric = output_df[numeric_columns].map(lambda x: not is_numeric(x))  # True for non-numeric entries

    # Step 5: Create invalid entries DataFrame including 'NO' and 'Komponen'
    invalid_entries = output_df[mask_non_numeric.any(axis=1)]  # Rows with any non-numeric entries
    invalid_entries.insert(0,'Tabel',sheet_name)

    return output_df, invalid_entries  # Return both valid DataFrame and invalid entries


In [43]:
def prepare_dataframe_5(super_sheet, sheet_name,year_to_check): 
    # Set the header based on the sheet name
    if sheet_name == '5a':
        header_array = [3, 4, 5]
    else:
        header_array = [2, 3, 4, 5]

    # Parse the sheet into a DataFrame
    input_df = super_sheet.parse(sheet_name, header=header_array)
    
    # Adjust the column names
    input_df.columns = input_df.columns.map(
        lambda x: tuple(
            str(x[i]).upper() if i == 0 and 'Unnamed' in str(x[1]) else x[i].strip() if isinstance(x[i], str) else x[i]
            for i in range(len(x))
        ) if isinstance(x, tuple) else x
    )

    # Step 1: Filter columns that have the year in the name (using regex)
    output_df = input_df.filter(regex=str(year_to_check))

    # Step 2: Insert 'NO' and 'Komponen' columns
    output_df.insert(0, 'NO', input_df[list(input_df.columns)[0]])
    output_df.insert(1, 'Komponen', input_df[list(input_df.columns)[1]])
    
    # Step 3: Remove rows after "keterangan"
    index_keterangan = output_df[output_df.apply(lambda row: row.astype(str).str.contains('keterangan', case=False).any(), axis=1)].index
    if not index_keterangan.empty:
        first_keterangan_index = index_keterangan[0]
        output_df = output_df.iloc[:first_keterangan_index]

    # Step 4: Apply numeric validation only to numeric columns
    numeric_columns = output_df.columns[2:]  # Only consider columns after 'NO' and 'Komponen'
    mask_non_numeric = output_df[numeric_columns].applymap(lambda x: not is_numeric(x))  # True for non-numeric entries

    # Step 5: Create invalid entries DataFrame including 'NO' and 'Komponen'
    invalid_entries = output_df[mask_non_numeric.any(axis=1)]  # Rows with any non-numeric entries

    # Step 6: Remove rows with any non-numeric entries from the output DataFrame
    output_df = output_df[~mask_non_numeric.any(axis=1)]  # Keep only valid entries

    return output_df, invalid_entries  # Return both valid DataFrame and invalid entries


In [44]:
def masking(df, key, value):
    return df[df[key] == value]

In [45]:
def average_formula(input_df,index,year_to_check):
    to_check = masking(input_df, "NO", index)
    to_check = to_check.iloc[:,3:]
    to_check = to_check.drop(to_check.columns[-1], axis=1)
    to_check_avg =np.ceil(float(to_check.mean(axis=1)))
    data_real = np.ceil(float(input_df.loc[input_df["NO"] == index, str(year_to_check)]))

    if to_check_avg == data_real:
        input_df = input_df.drop(index-1)
    
    return input_df

In [46]:
def sum_formula(input_df,index,year_to_check):
    to_check = masking(input_df, "NO", index)
    to_check = to_check.iloc[:,3:]
    to_check = to_check.drop(to_check.columns[-1], axis=1)
    to_check_avg =np.ceil(float(to_check.sum(axis=1)))
    data_real = np.ceil(float(input_df.loc[input_df["NO"] == index, str(year_to_check)]))

    if to_check_avg == data_real:
        input_df = input_df.drop(index-1)
    
    return input_df

In [47]:
def h1_score_year_month(input_df, sheet_name):
    year_to_check = datetime.date.today().year - 1
    year_str = str(year_to_check)

    # Get two arrays for comparison
    arr1 = pd.to_numeric(input_df.filter(like=year_str).iloc[:, 0], errors='coerce').to_numpy()

    # Initialize arr2
    arr2 = np.zeros_like(arr1)  # Default arr2 to zeros if not overridden

    # Check for Dec column or fallback to Q4
    if sheet_name == "13":
        # Select the relevant columns
        to_check = input_df.iloc[:, 3:]

        # Replace '-' with NaN, then convert to numeric
        to_check.replace('-', np.nan, inplace=True)
        to_check = to_check.apply(pd.to_numeric, errors='coerce').fillna(0)

        # Calculate the mean and round it up
        mean_values = np.ceil(to_check.mean(axis=1))

        # Convert the mean values to a numpy array
        arr2 = mean_values.to_numpy()
    else:    
        if (year_to_check, 'Dec') in input_df.columns:
            arr2 = pd.to_numeric(input_df[(year_to_check, 'Dec')], errors='coerce').to_numpy()
        elif (year_to_check, 'Des') in input_df.columns:
            arr2 = pd.to_numeric(input_df[(year_to_check, 'Des')], errors='coerce').to_numpy()
        else:
            arr2 = pd.to_numeric(input_df.filter(like='Q4').iloc[:, 0], errors='coerce').to_numpy()

    # Mask for valid and non-NaN values
    valid_mask = ~np.isnan(arr1) & ~np.isnan(arr2)

    # Find mismatches where arr1 != arr2
    mismatches = valid_mask & (arr1 != arr2)

    # Calculate differences for mismatches
    differences = np.where(mismatches, np.abs(arr1 - arr2), 0)

    # Add the H1 Score column with the differences (no filtering)
    input_df['H1 SCORE'] = differences

    # HARDCODED HARDCODED HARDCODED
    if sheet_name == "1":
        input_df = average_formula(input_df, 34,year_to_check)
        input_df = sum_formula(input_df, 35,year_to_check)

        ignore_array = [2, 5, 28, 29, 30]
        for item in ignore_array:
            input_df.loc[item - 1, 'H1 SCORE'] = "TBC"
    if sheet_name == "2":
        input_df.replace('-', 0, inplace=True)
        sum_array = [39, 40, 42,64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]
        for item in sum_array:
            input_df = sum_formula(input_df, item,year_to_check)

        # ignore_array = [1, 17, 38]
        # for item in ignore_array:
        #     input_df.loc[item - 1, 'H1 SCORE'] = "TBC"
    if sheet_name == "13":
        input_df.replace('-', 0, inplace=True)
        sum_array = [37, 39, 40, 41]
        for item in sum_array:
            input_df = sum_formula(input_df, item,year_to_check)
    if sheet_name == "14":
        input_df.replace('-', 0, inplace=True)

        average_array = [31, 33, 38, 42, 43, 44, 45, 46, 47]
        for item in average_array:
            input_df = average_formula(input_df, item,year_to_check)
        
        sum_array = [35, 36, 37]
        for item in sum_array:
            input_df = sum_formula(input_df, item,year_to_check)

        ignore_array = [32, 34, 39]
        for item in ignore_array:
            input_df.loc[item - 1, 'H1 SCORE'] = "TBC"

    # Create a DataFrame containing only the rows with mismatches
    mismatch_df = input_df.loc[input_df['H1 SCORE'] != 0] 

    # Return the entire dataframe and the mismatch dataframe
    return input_df, mismatch_df

In [48]:
def h1_score_claim_liabilities(input_df, sheet_name):
    if sheet_name != "5a":
        last_level_columns = input_df.columns.get_level_values(-1)
        unique_subcategories = np.array(last_level_columns.unique())[1:]  
    
    middle_level_columns = input_df.columns.get_level_values(2)
    unique_categories = np.array(middle_level_columns.unique())[1:]

    # Loop through all the unique categories (Claims, Liabilities) and their subcategories
    for cat in unique_categories:
        if sheet_name == "5a":
            # Select the first and last columns matching the category
            arr1 = pd.to_numeric(input_df.filter(like=f'{cat}').iloc[:, 0], errors='coerce').fillna(0).to_numpy()
            arr2 = pd.to_numeric(input_df.filter(like=f'{cat}').iloc[:, -1], errors='coerce').fillna(0).to_numpy()

            # Calculate mismatches where the values are different
            mismatches = (arr1 != arr2)

            # Calculate the absolute difference where mismatches occur, otherwise fill with 0
            differences = np.where(mismatches, np.abs(arr1 - arr2), 0)

            # Assign the differences to a new column in the dataframe
            input_df[f'{cat} H1 SCORE'] = differences
        else:
            for item in unique_subcategories:
                # Select the first and last columns matching the category and subcategory
                arr1 = pd.to_numeric(input_df.filter(like=f'{cat}').filter(like=item).iloc[:, 0], errors='coerce').fillna(0).to_numpy()
                arr2 = pd.to_numeric(input_df.filter(like=f'{cat}').filter(like=item).iloc[:, -1], errors='coerce').fillna(0).to_numpy()

                # Calculate mismatches where the values are different
                mismatches = (arr1 != arr2)

                # Calculate the absolute difference where mismatches occur, otherwise fill with 0
                differences = np.where(mismatches, np.abs(arr1 - arr2), 0)

                # Assign the differences to a new column in the dataframe
                input_df[f'{cat} {item} H1 SCORE'] = differences

    # Use filter(regex=...) to capture all columns with 'H1 SCORE' in their names
    score_columns = input_df.filter(regex='H1 SCORE')

    # # Print the score_columns for debugging
    # print(score_columns)

    # Filter the input_df where any of the H1 SCORE columns have non-zero values
    mismatch_df = input_df[(score_columns != 0).any(axis=1)]

    return input_df, mismatch_df

In [49]:
def run_horizontal_check(super_sheet, sheet_to_check):
    year_to_check = datetime.date.today().year - 1
    if sheet_to_check.startswith("5"):
        start_df, invalid = prepare_dataframe_5(super_sheet, sheet_to_check,year_to_check)
        final, clean = h1_score_claim_liabilities(start_df, sheet_to_check)
    else:
        start_df, invalid = prepare_dataframe(super_sheet, sheet_to_check,year_to_check)
        final, clean = h1_score_year_month(start_df, sheet_to_check)

    final.columns = [' '.join(str(item).strip().upper() for item in col if item is not None) for col in final.columns]

    # Remove the entire "UNNAMED: <number>_LEVEL_1" component from headers
    final.columns = [re.sub(r'UNNAMED:\s*\d+(_LEVEL_1)?', '', col).strip() for col in final.columns]

    clean.columns = [' '.join(str(item).strip().upper() for item in col if item is not None) for col in final.columns]

    # Remove the entire "UNNAMED: <number>_LEVEL_1" component from headers
    clean.columns = [re.sub(r'UNNAMED:\s*\d+(_LEVEL_1)?', '', col).strip() for col in final.columns]

    # Combine multi-level headers into a single header
    invalid.columns = [' '.join(str(item).strip().upper() for item in col if item is not None) for col in invalid.columns]

    # Remove the entire "UNNAMED: <number>_LEVEL_1" component from headers
    invalid.columns = [re.sub(r'UNNAMED:\s*\d+(_LEVEL_1)?', '', col).strip() for col in invalid.columns]

    # # Ensure the output folder exists, create it if necessary
    # FILEPATH = os.path.join(os.getcwd(), "SAVED")
    # if not os.path.exists(FILEPATH):
    #     os.makedirs(FILEPATH)

    # output_filename = os.path.join(FILEPATH, f'SSKI TABLE {sheet_to_check} LOG.xlsx')
    
    # # Check if the Excel file exists
    # if os.path.exists(output_filename):
    #     wb = openpyxl.load_workbook(output_filename)

    #     if 'Horizontal Check' in wb.sheetnames:
    #         sheet = wb['Horizontal Check']
    #         for row in sheet.iter_rows(min_row=1, max_row=sheet.max_row):
    #             for cell in row:
    #                 cell.value = None
    #     else:
    #         sheet = wb.create_sheet('Horizontal Check')
    # else:
    #     wb = openpyxl.Workbook()
    #     sheet = wb.active
    #     sheet.title = 'Horizontal Check'

    # # Write the header to the "Horizontal Check" sheet
    # for c_idx, column_name in enumerate(final.columns, 1):
    #     # Convert tuple column names to string if necessary
    #     if isinstance(column_name, tuple):
    #         column_name = ' '.join(map(str, column_name))  # Convert to space-separated string
    #     else:
    #         column_name = str(column_name)  # Ensure it's a string if it's not a tuple
    #     sheet.cell(row=1, column=c_idx, value=column_name)

    # # Function to safely convert values to string
    # def convert_value(value):
    #     if isinstance(value, (tuple, list)):
    #         return ', '.join(map(str, value))
    #     elif pd.isna(value):
    #         return ''
    #     else:
    #         return str(value)

    # # Write the DataFrame rows (excluding header) to the sheet
    # for r_idx, row in enumerate(final.itertuples(index=False), 2):
    #     for c_idx, value in enumerate(row, 1):
    #         sheet.cell(row=r_idx, column=c_idx, value=convert_value(value))

    # # Define the thin border style
    # thin_border = Border(
    #     left=Side(style='thin'),
    #     right=Side(style='thin'),
    #     top=Side(style='thin'),
    #     bottom=Side(style='thin')
    # )

    # # Set a default font for the header and data
    # header_font = Font(bold=True)
    # data_font = Font()
    # center_alignment = Alignment(wrap_text=True, horizontal='center', vertical='center')

    # # Apply styles to the header row
    # for cell in sheet[1]:
    #     cell.font = header_font
    #     cell.alignment = center_alignment
    #     cell.border = thin_border

    # # Apply styles to the data rows
    # for row in sheet.iter_rows(min_row=2, max_row=sheet.max_row):
    #     for cell in row:
    #         cell.font = data_font
    #         cell.alignment = center_alignment
    #         cell.border = thin_border

    # # Define the column widths
    # column_widths = {
    #     'A': 5, 'B': 60
    # }

    # # Apply column widths
    # for col, width in column_widths.items():
    #     sheet.column_dimensions[col].width = width

    # # 
    # # 
    # #  the workbook with the updated data
    # wb.save(output_filename)
    # return final, clean, invalid, f"Data has been written to weird"
    return final, clean, invalid, f"Horizontal Algorithm Done Processing"


In [50]:
def set_border(sheet, cell_range):
    thin = Side(border_style="thin", color="000000")
    for row in sheet[cell_range]:
        for cell in row:
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)


In [51]:
def save_ver(df_raw):
    # Extract the number after 'SSKI.' from the "Path" column
    extracted_numbers = df_raw['Path'].str.extract(r'SSKI\.([^\.]+)\.[^\.]*')[0]

    # Set up the file path
    if os.path.exists("./SAVED"):
        FILEPATH = os.getcwd() + "/SAVED"
    else:
        FILEPATH = os.path.join(os.getcwd(), "SAVED")
        os.makedirs(FILEPATH, exist_ok=True)  # Create the directory if it doesn't exist

    # Extract the number for the filename
    num = extracted_numbers.dropna().tolist()[0] if not extracted_numbers.empty else None
    if num == '5d1':
        num = '5d.1'
    elif num == '5d2':
        num = '5.d.2'

    # File path for saving
    output_filename = os.path.join(FILEPATH, f'SSKI TABLE {num} LOG.xlsx')

    # Load the existing workbook or create a new one
    if os.path.exists(output_filename):
        wb = load_workbook(output_filename)
        # Create 'VERTICAL CHECK' sheet if it doesn't exist
        if 'Vertical Check' in wb.sheetnames:
            sheet = wb['Vertical Check']
        else:
            sheet = wb.create_sheet(title='Vertical Check')
    else:
        wb = Workbook()
        sheet = wb.active
        sheet.title = 'Vertical Check'

    # Find the next empty row
    next_row = sheet.max_row + 2  # This will point to the row after the last filled row

    # Write the headers for the current DataFrame
    for c_idx, value in enumerate(df_raw.columns, start=1):
        sheet.cell(row=next_row, column=c_idx, value=value)

    # Set the width for the first and second columns only and wrap the text
    for col_idx in [1, 2]:  # Only first two columns
        col_letter = get_column_letter(col_idx)
        sheet.column_dimensions[col_letter].width = 40
        for row in sheet.iter_rows(min_row=next_row, max_row=sheet.max_row, min_col=col_idx, max_col=col_idx):
            for cell in row:
                cell.alignment = Alignment(wrap_text=True)

    # Move to the next row for data
    next_row += 1

    # Append the new DataFrame below the existing data
    for r_idx, row in df_raw.iterrows():
        for c_idx, value in enumerate(row, start=1):  # Start from column 1 (A)
            cell = sheet.cell(row=next_row, column=c_idx, value=value)
            if c_idx in [1, 2]:  # Apply wrapping to the first two columns
                cell.alignment = Alignment(wrap_text=True)
        next_row += 1  # Move to the next row after writing each row of the DataFrame

    # Apply borders to the table (from headers to last row)
    total_rows = len(df_raw) + 1  # +1 to include header
    cell_range = f"A{next_row - total_rows}:{get_column_letter(len(df_raw.columns))}{next_row - 1}"
    set_border(sheet, cell_range)
    wb.save(output_filename)

In [52]:
def set_border(sheet, cell_range):
    from openpyxl.styles import Border, Side
    thin_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                        top=Side(style='thin'), bottom=Side(style='thin'))

    rows = sheet[cell_range]
    for row in rows:
        for cell in row:
            cell.border = thin_border

In [53]:
def getrecap_data(clean_data, clean_keys_list, raw_data, raw_keys_list, hor_raw_keys_list, horizontal_raw_data, horizontal_clean_data,current_year):
    error_counts = {}
    ver_error_count = 0
    ver_total_count = 0

    for i in range(len(clean_data)):
        df_clean = pd.DataFrame(clean_data[clean_keys_list[i]])
        df_raw = pd.DataFrame(raw_data[raw_keys_list[i]])

        sski_path = df_clean['Path'][0]  # Assuming 'Path' column exists
        sski_number = sski_path.split('.')[1]  # Extract the number after 'SSKI'

        column_count_error = len(df_clean.columns) - 2
        column_count_total = len(df_raw.columns) - 2
        column_count_correct = column_count_total - column_count_error  # Calculate correct count

        ver_error_count += column_count_error
        ver_total_count += column_count_total

        if sski_number not in error_counts:
            error_counts[sski_number] = {"Vertikal - Jumlah Selisih": 0, "Vertikal - Jumlah Benar": 0, "Vertikal - Jumlah Total": 0}

        error_counts[sski_number]["Vertikal - Jumlah Selisih"] += column_count_error
        error_counts[sski_number]["Vertikal - Jumlah Benar"] += column_count_correct
        error_counts[sski_number]["Vertikal - Jumlah Total"] += column_count_total

    total_error_rows = 0
    total_correct_rows = 0
    total_rows = 0
    hor_error = {}

    for item in hor_raw_keys_list:
        final = pd.DataFrame(horizontal_raw_data[item])
        
        if final is not None and not final.empty:
            total_rows_in_table = len(final)  # Total rows in the table
            correct_rows = total_rows_in_table  # Assume all rows are correct initially

            if item in horizontal_clean_data:
                clean = pd.DataFrame(horizontal_clean_data[item])
                error_rows = len(clean)  # Error rows come from `clean`
                correct_rows = total_rows_in_table - error_rows  # Correct rows are the difference
            else:
                error_rows = 0  # If there's no `clean` data, assume no errors

            hor_error[item] = {
                "Horizontal - Jumlah Selisih": error_rows,
                "Horizontal - Jumlah Benar": correct_rows,
                "Horizontal - Jumlah Total": total_rows_in_table
            }

            total_error_rows += error_rows
            total_correct_rows += correct_rows
            total_rows += total_rows_in_table


    df_hor_error = pd.DataFrame.from_dict(hor_error, orient='index').reset_index()
    df_hor_error.rename(columns={'index': 'table_name'}, inplace=True)

    df_error_counts = pd.DataFrame.from_dict(error_counts, orient='index').reset_index()
    df_error_counts.rename(columns={'index': 'table_name'}, inplace=True)

    df_combined = pd.merge(df_hor_error, df_error_counts, on='table_name', how='outer')

    df_combined.fillna(0, inplace=True)

    numeric_columns = df_combined.select_dtypes(include=['float64']).columns
    df_combined[numeric_columns] = df_combined[numeric_columns].astype(int)

    df_combined['table_number'] = df_combined['table_name'].str.extract('(\d+)').astype(int)
    df_combined.sort_values(by='table_number', inplace=True)

    df_combined.drop(columns=['table_number'], inplace=True)

    merge_pairs = {
        '5d.1': '5d1',
        '5d2': '5.d.2'
    }

    # Iterate through the DataFrame and sum the relevant rows
    for table, merge_with in merge_pairs.items():
        row_to_merge = df_combined[df_combined['table_name'] == table]
        row_to_merge_with = df_combined[df_combined['table_name'] == merge_with]

        if not row_to_merge.empty and not row_to_merge_with.empty:
            merged_index = row_to_merge.index[0]  # Use the index of the first row
            # Sum the values and assign to the existing row
            df_combined.loc[merged_index, df_combined.columns[2:]] += row_to_merge_with.iloc[0, 2:]
            # Remove the merged row
            df_combined = df_combined.drop(row_to_merge_with.index)

    # Reset index for clarity (optional)
    display(df_combined)
    df_combined.reset_index(drop=True, inplace=True)
    display(df_combined)
    
    last_period = f"{month} {current_year}"
    df_combined['Last Period'] = last_period

    # Display the final merged DataFrame
    return df_combined

In [54]:
def Before_After(df_cur, df_prev):

    # Create copies of the DataFrames to avoid modifying the originals
    df_cur_copy = df_cur.copy()
    df_prev_copy = df_prev.copy()
    
    # Perform an outer join on 'Komponen' and 'NO' to align both DataFrames, even if they are non-identical
    merged_df = df_cur_copy.merge(df_prev_copy, on=['Komponen', 'NO'], how='outer', suffixes=('_Terkini', '_Terdahulu'))
    
    # Identify columns to compare by checking for suffixes and excluding 'Komponen' and 'NO'
    comparison_columns = [col.replace('_Terkini', '') for col in merged_df.columns if '_Terkini' in col]
    
    # Initialize an empty DataFrame to store comparison results
    comparison_df = pd.DataFrame()
    
    # Add 'Komponen' and 'NO' columns first
    comparison_df['NO'] = merged_df['NO']
    comparison_df['Komponen'] = merged_df['Komponen']
    
    # Loop through each attribute to create separate columns for "Periode Terkini" and "Periode Terdahulu"
    for col in comparison_columns:
        terkini_col = col + '_Terkini'
        terdahulu_col = col + '_Terdahulu'
        
        # Add columns for each period and fill with "✓" if values match, or the actual values if they differ
        comparison_df[(col, 'Periode Terkini')] = merged_df[terkini_col].where(merged_df[terkini_col] != merged_df[terdahulu_col], '✓')
        comparison_df[(col, 'Periode Terdahulu')] = merged_df[terdahulu_col].where(merged_df[terkini_col] != merged_df[terdahulu_col], '✓')
    
    # Drop rows where all values (excluding 'Komponen' and 'NO') are '✓' (indicating no differences in any column)
    non_index_cols = comparison_df.columns.difference(['Komponen', 'NO'])
    comparison_df = comparison_df.loc[~(comparison_df[non_index_cols] == '✓').all(axis=1)]
    comparison_df = comparison_df.dropna(how='all', subset=non_index_cols)
    comparison_df = comparison_df.loc[~comparison_df[non_index_cols].apply(lambda row: set(row.dropna()) <= {'✓'}, axis=1)]

    # Sort the comparison DataFrame by 'NO'
    comparison_df = comparison_df.sort_values(by='NO', ascending=True)
    
    # Return the comparison DataFrame or None if empty
    return comparison_df if not comparison_df.empty else None


In [ ]:
def main(list_tahun, indikator_asli, indikator_calculated, mappings, kelompok):
    data_clean_array = {}
    data_raw_array = {}
    data_summary_array = {}
    data_horizontal_array = {}
    data_raw_hor_array = {}
    data_bef_aft = {}

    for i in range(len(indikator_asli)):
        # Combine indicators and skip if invalid
        df_clean, df_raw, df_summary = gabungkan_dataframe_2bag(list_tahun, indikator_asli[i], indikator_calculated[i], mappings)
        # save_ver(df_raw)
        
        sski_path = df_clean['Path'][0]  # Assuming 'Path' column exists
        sski_number = sski_path.split('.')[1]

        json_raw_df = df_raw.to_json(orient='records')
        data_raw_array[f"{sski_number}-{kelompok[i]}"] = json.loads(json_raw_df)

        json_clean_df = df_clean.to_json(orient='records')
        data_clean_array[f"{sski_number}-{kelompok[i]}"] = json.loads(json_clean_df)

        json_summary_df = df_summary.to_json(orient='records')
        data_summary_array[f"{sski_number}-{kelompok[i]}"] = json.loads(json_summary_df)
    
    # Horizontal check table list and error counts
    table_list = ["1", "2", "3", "4", "5a", "5b", "5c", "5d", "5d.1", "5.d.2", "6", "7", "8", "9", "10", "11a", "12", "13", "14", "15", "16a", "17", "18", "19", "20"]
    horizontal_results = {}

    # Assuming you still want to run horizontal checks
    for item in table_list:
        final, clean, invalid, msg = run_horizontal_check(super_sheet_new, item)
        if clean is not None and not clean.empty:
            json_horizontal_df = clean.to_json(orient='records')
            data_horizontal_array[item] = json.loads(json_horizontal_df)  # Save clean DataFrame for later display
        json_horizontal_raw_df = final.to_json(orient='records')
        data_raw_hor_array[item] = json.loads(json_horizontal_raw_df)

    year_to_check = "20"
    for item in table_list:
        if item == "5a" or item == "5b" or item == "5c" or item == "5d" or item == "5d.1" or item == "5.d.2":
            old_df, invalid_entries = prepare_dataframe_5(super_sheet_old,item,year_to_check)
            new_df, invalid_entries = prepare_dataframe_5(super_sheet_new,item,year_to_check)
        else :
            old_df, invalid_entries = prepare_dataframe(super_sheet_old,item,year_to_check)
            new_df, invalid_entries = prepare_dataframe(super_sheet_new,item,year_to_check)

        df = Before_After(new_df, old_df)
        if df is not None:
            json_bef_aft_df = df.to_json(orient='records')
            data_bef_aft[item] = json.loads(json_bef_aft_df)

    log_data = {
        "created_at" : datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "file_name": file_path,
        "period": f"{month} {year}"
    }

    combined_data = {
        "log_data": log_data,
        "clean_data": data_clean_array,
        "raw_data": data_raw_array,
        "summary_data": data_summary_array,
        "horizontal_clean_data": data_horizontal_array,
        "horizontal_raw_data": data_raw_hor_array,
        "before_after_data": data_bef_aft
    }

    current_date = datetime.datetime.now()
    current_year = current_date.year

    csv_recap = getrecap_data(
        combined_data["clean_data"], 
        list(combined_data["clean_data"].keys()), 
        combined_data["raw_data"], 
        list(combined_data["raw_data"].keys()), 
        list(combined_data["horizontal_raw_data"].keys()), 
        combined_data["horizontal_raw_data"], 
        combined_data["horizontal_clean_data"],
        current_year
    )

    csv_recap.insert(0, 'No.', range(1, len(csv_recap) + 1))
    csv_recap.to_csv('/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/data/data_rekap_konsistensi.csv', sep=';', index=False)
    csv_recap = csv_recap.to_json(orient='records')

    combined_data['recap_download'] = csv_recap

    output_file_path = '/Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/data/data_sski.json'

    with open(output_file_path, 'w') as json_file:
        json.dump(combined_data, json_file, indent=4)

    print(f"Data has been written to {output_file_path}")

In [59]:
# Example parameters (replace with your actual data)
list_tahun = ["2015","2016","2017","2018","2019","2020","2021","2022","2023","2024","2025","2026"]

# Assuming indikator_asli, indikator_calculated, mappings, and kelompok are defined
main(list_tahun, indikator_asli, indikator_calculated, mappings, kelompok)

/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_68476/1429410141.py:136: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(data_frames, ignore_index=True)
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_68476/1419689938.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_gabungan = pd.concat([df1_filtered_common, df2_filtered_common], ignore_index=True)
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_68476/1419689938.py:107: PerformanceWar

,table_name,Horizontal - Jumlah Selisih,Horizontal - Jumlah Benar,Horizontal - Jumlah Total,Vertikal - Jumlah Selisih,Vertikal - Jumlah Benar,Vertikal - Jumlah Total
0,1,8,51,59,0,0,0
11,2,6,77,83,10,770,780
13,3,0,308,308,2,570,572
14,4,0,216,216,0,987,987
22,5d2,0,42,42,70,330,400
20,5d.1,0,42,42,0,200,200
19,5d,0,42,42,0,300,300
18,5c,0,42,42,0,400,400
17,5b,0,15,15,0,800,800
16,5a,0,15,15,0,300,300


,table_name,Horizontal - Jumlah Selisih,Horizontal - Jumlah Benar,Horizontal - Jumlah Total,Vertikal - Jumlah Selisih,Vertikal - Jumlah Benar,Vertikal - Jumlah Total
0,1,8,51,59,0,0,0
1,2,6,77,83,10,770,780
2,3,0,308,308,2,570,572
3,4,0,216,216,0,987,987
4,5d2,0,42,42,70,330,400
5,5d.1,0,42,42,0,200,200
6,5d,0,42,42,0,300,300
7,5c,0,42,42,0,400,400
8,5b,0,15,15,0,800,800
9,5a,0,15,15,0,300,300


Data has been written to /Users/ferroyudisthira/Desktop/DSTA_DQAD/V&H_Check/data/data_sski.json


In [57]:
old_df, invalid_entries = prepare_dataframe(super_sheet_old,"1","20")
new_df, invalid_entries = prepare_dataframe(super_sheet_new,"1","20")

df = Before_After(new_df, old_df)

df

/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_68476/2870321726.py:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  comparison_df[(col, 'Periode Terkini')] = merged_df[terkini_col].where(merged_df[terkini_col] != merged_df[terdahulu_col], '✓')
/var/folders/92/cpb0j1l94536c_zzljx18zw80000gn/T/ipykernel_68476/2870321726.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  comparison_df[(col, 'Periode Terdahulu')] = merged_df[terdahulu_col].where(merged_df[terkini_col] != merged_df[terdahulu_col], '✓')
/var/folders/92/